In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch import Tensor
from phenom_p import IMRPhenomPv2
params={
    'text.usetex':True,
    'font.family':'serif',
    'xtick.minor.visible':True,
    'ytick.minor.visible':True,
    'xtick.top':True,
    'ytick.left':True,
    'ytick.right':True,
    'xtick.direction':'out',
    'ytick.direction':'out',
    'xtick.minor.size':2.5,
    'xtick.major.size':5,
    'ytick.minor.size':2.5,
    'ytick.major.size':5,
    'axes.axisbelow':True,
    "xtick.labelsize": 16,
    "ytick.labelsize": 16,
    "axes.labelsize": 18,
    "legend.fontsize": 14,
    "legend.title_fontsize": 16,
    "figure.titlesize": 22,
    "axes.labelpad": 10.0,
}
plt.rcParams.update(params)

In [2]:
phenom_p = IMRPhenomPv2().to('cuda')
chi1z = 0.5
chi2z = -0.1
mass_ratio = 1.5
chirp_mass = 30
distance = 100
tc = 0
phic = 0
inclination = 0
torch_freqs = torch.arange(
        10, 300, 1/128.
    ).to('cuda')
_params = torch.tensor(
        [
            chirp_mass,
            mass_ratio,
            0,
            0,
            chi1z,
            0,
            0,
            chi2z,
            distance,
            tc,
            phic,
            inclination,
        ]
    ).repeat(10, 1).to('cuda')
f_ref = 20.
batched_chirp_mass = _params[:, 0]
batched_mass_ratio = _params[:, 1]
batched_chi1x = _params[:, 2]
batched_chi1y = _params[:, 3]
batched_chi1z = _params[:, 4]
batched_chi2x = _params[:, 5]
batched_chi2y = _params[:, 6]
batched_chi2z = _params[:, 7]
batched_distance = _params[:, 8]
batched_tc = _params[:, 9]
batched_phic = _params[:, 10]
batched_inclination = _params[:, 11]

In [6]:
hp_torch, hc_torch = phenom_p.forward(
        torch_freqs,
        batched_chirp_mass,
        batched_mass_ratio,
        batched_chi1x,
        batched_chi1y,
        batched_chi1z,
        batched_chi2x,
        batched_chi2y,
        batched_chi2z,
        batched_distance,
        batched_tc,
        batched_phic,
        batched_inclination,
        f_ref,
    )
hp_torch.device, hc_torch.device

(device(type='cuda', index=0), device(type='cuda', index=0))